## Setup

In [4]:
import sys, os, json, csv

import numpy as np
import pandas as pd

### Data

In [23]:
# df = pd.read_csv('data/AB_NYC_2019.csv')  
# df = pd.read_csv('data/AB_US_2020.csv')  
df = pd.read_csv('data/airbnb-listings.csv', sep=';', low_memory=False)

# EDA

In [24]:
df.columns

Index(['ID', 'Listing Url', 'Scrape ID', 'Last Scraped', 'Name', 'Summary',
       'Space', 'Description', 'Experiences Offered', 'Neighborhood Overview',
       'Notes', 'Transit', 'Access', 'Interaction', 'House Rules',
       'Thumbnail Url', 'Medium Url', 'Picture Url', 'XL Picture Url',
       'Host ID', 'Host URL', 'Host Name', 'Host Since', 'Host Location',
       'Host About', 'Host Response Time', 'Host Response Rate',
       'Host Acceptance Rate', 'Host Thumbnail Url', 'Host Picture Url',
       'Host Neighbourhood', 'Host Listings Count',
       'Host Total Listings Count', 'Host Verifications', 'Street',
       'Neighbourhood', 'Neighbourhood Cleansed',
       'Neighbourhood Group Cleansed', 'City', 'State', 'Zipcode', 'Market',
       'Smart Location', 'Country Code', 'Country', 'Latitude', 'Longitude',
       'Property Type', 'Room Type', 'Accommodates', 'Bathrooms', 'Bedrooms',
       'Beds', 'Bed Type', 'Amenities', 'Square Feet', 'Price', 'Weekly Price',
       'Month

In [25]:
df.head(3)

,ID,Listing Url,Scrape ID,Last Scraped,Name,Summary,Space,Description,Experiences Offered,Neighborhood Overview,...,Review Scores Communication,Review Scores Location,Review Scores Value,License,Jurisdiction Names,Cancellation Policy,Calculated host listings count,Reviews per Month,Geolocation,Features
0,17619081,https://www.airbnb.com/rooms/17619081,20170615002708,2017-06-15,Nice smallroom i central old Frederiksberg,Min bolig lægger tæt på Bus og Metro. Frederik...,NaN,Min bolig lægger tæt på Bus og Metro. Frederik...,none,NaN,...,10.0,8.0,10.0,NaN,NaN,flexible,2.0,0.66,"55.6759793551,12.5329597787","Host Is Superhost,Host Has Profile Pic,Host Id..."
1,13356620,https://www.airbnb.com/rooms/13356620,20170615002708,2017-06-15,Spacious 95 m2 3 room apt. with great location,Spacious and sunny apt. located close to the b...,The apt. is located on 3rd floor and faces a t...,Spacious and sunny apt. located close to the b...,none,NaN,...,10.0,10.0,9.0,NaN,NaN,strict,1.0,0.41,"55.6759094982,12.5343258647","Host Has Profile Pic,Host Identity Verified,Is..."
2,18993602,https://www.airbnb.com/rooms/18993602,20170615002708,2017-06-15,Bright and lovely 3 room apartment in Copenhagen,Lovely newly renovated 3 room apartment locate...,NaN,Lovely newly renovated 3 room apartment locate...,none,NaN,...,NaN,NaN,NaN,NaN,NaN,moderate,2.0,NaN,"55.6914472628,12.5328311114","Host Has Profile Pic,Host Identity Verified"


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 494954 entries, 0 to 494953
Data columns (total 89 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   ID                              494954 non-null  object 
 1   Listing Url                     494954 non-null  object 
 2   Scrape ID                       494954 non-null  object 
 3   Last Scraped                    494953 non-null  object 
 4   Name                            494536 non-null  object 
 5   Summary                         478753 non-null  object 
 6   Space                           347019 non-null  object 
 7   Description                     494794 non-null  object 
 8   Experiences Offered             494954 non-null  object 
 9   Neighborhood Overview           291604 non-null  object 
 10  Notes                           197643 non-null  object 
 11  Transit                         303786 non-null  object 
 12  Access          

# Overview

In this project, data for Airbnb listings is explored in an effort to glean insights relating to rental properties. An exhaustive hypotheses testing and modeling process will be performed in order to offer guidance to property owners renting through Airbnb or similar platforms.

## About the Datasets

The datasets used in this project were retrieved from various sources, listed below.  

  [Airbnb — Listings](https://public.opendatasoft.com/explore/dataset/airbnb-listings/information/?disjunctive.host_verifications&disjunctive.amenities&disjunctive.features&dataChart=eyJxdWVyaWVzIjpbeyJjaGFydHMiOlt7InR5cGUiOiJjb2x1bW4iLCJmdW5jIjoiQ09VTlQiLCJ5QXhpcyI6Imhvc3RfbGlzdGluZ3NfY291bnQiLCJzY2llbnRpZmljRGlzcGxheSI6dHJ1ZSwiY29sb3IiOiJyYW5nZS1jdXN0b20ifV0sInhBeGlzIjoiY2l0eSIsIm1heHBvaW50cyI6IiIsInRpbWVzY2FsZSI6IiIsInNvcnQiOiIiLCJzZXJpZXNCcmVha2Rvd24iOiJyb29tX3R5cGUiLCJjb25maWciOnsiZGF0YXNldCI6ImFpcmJuYi1saXN0aW5ncyIsIm9wdGlvbnMiOnsiZGlzanVuY3RpdmUuaG9zdF92ZXJpZmljYXRpb25zIjp0cnVlLCJkaXNqdW5jdGl2ZS5hbWVuaXRpZXMiOnRydWUsImRpc2p1bmN0aXZlLmZlYXR1cmVzIjp0cnVlfX19XSwidGltZXNjYWxlIjoiIiwiZGlzcGxheUxlZ2VuZCI6dHJ1ZSwiYWxpZ25Nb250aCI6dHJ1ZX0%3D)  

  [Kaggle — New York City Airbnb Open Data](https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data)  
  Airbnb listings and metrics in NYC, NY, USA (2019)  
  
  - Since 2008, guests and hosts have used Airbnb to expand on traveling possibilities and present more unique, personalized way of experiencing the world. This dataset describes the listing activity and metrics in NYC, NY for 2019.  
  
  
  - This data file includes all needed information to find out more about hosts, geographical availability, necessary metrics to make predictions and draw conclusions.  
  
  [Kaggle — U.S. Airbnb Open Data](https://www.kaggle.com/kritikseth/us-airbnb-open-data)  
  Airbnb listings and metrics 

## Bibliography

All Datasets used have been made available under Creative Commons CC0 1.0 Universal (CC0 1.0) "Public Domain Dedication"
  
  
  
  [Airbnb — Listings](https://public.opendatasoft.com/explore/dataset/airbnb-listings/information/?disjunctive.host_verifications&disjunctive.amenities&disjunctive.features&dataChart=eyJxdWVyaWVzIjpbeyJjaGFydHMiOlt7InR5cGUiOiJjb2x1bW4iLCJmdW5jIjoiQ09VTlQiLCJ5QXhpcyI6Imhvc3RfbGlzdGluZ3NfY291bnQiLCJzY2llbnRpZmljRGlzcGxheSI6dHJ1ZSwiY29sb3IiOiJyYW5nZS1jdXN0b20ifV0sInhBeGlzIjoiY2l0eSIsIm1heHBvaW50cyI6IiIsInRpbWVzY2FsZSI6IiIsInNvcnQiOiIiLCJzZXJpZXNCcmVha2Rvd24iOiJyb29tX3R5cGUiLCJjb25maWciOnsiZGF0YXNldCI6ImFpcmJuYi1saXN0aW5ncyIsIm9wdGlvbnMiOnsiZGlzanVuY3RpdmUuaG9zdF92ZXJpZmljYXRpb25zIjp0cnVlLCJkaXNqdW5jdGl2ZS5hbWVuaXRpZXMiOnRydWUsImRpc2p1bmN0aXZlLmZlYXR1cmVzIjp0cnVlfX19XSwidGltZXNjYWxlIjoiIiwiZGlzcGxheUxlZ2VuZCI6dHJ1ZSwiYWxpZ25Nb250aCI6dHJ1ZX0%3D)  
  - Dataset available under Creative Commons CC0: Public Domain License, 2020  
  - [Available from public.opendatasoft.com](https://public.opendatasoft.com/explore/dataset/airbnb-listings/information/?disjunctive.host_verifications&disjunctive.amenities&disjunctive.features&dataChart=eyJxdWVyaWVzIjpbeyJjaGFydHMiOlt7InR5cGUiOiJjb2x1bW4iLCJmdW5jIjoiQ09VTlQiLCJ5QXhpcyI6Imhvc3RfbGlzdGluZ3NfY291bnQiLCJzY2llbnRpZmljRGlzcGxheSI6dHJ1ZSwiY29sb3IiOiJyYW5nZS1jdXN0b20ifV0sInhBeGlzIjoiY2l0eSIsIm1heHBvaW50cyI6IiIsInRpbWVzY2FsZSI6IiIsInNvcnQiOiIiLCJzZXJpZXNCcmVha2Rvd24iOiJyb29tX3R5cGUiLCJjb25maWciOnsiZGF0YXNldCI6ImFpcmJuYi1saXN0aW5ncyIsIm9wdGlvbnMiOnsiZGlzanVuY3RpdmUuaG9zdF92ZXJpZmljYXRpb25zIjp0cnVlLCJkaXNqdW5jdGl2ZS5hbWVuaXRpZXMiOnRydWUsImRpc2p1bmN0aXZlLmZlYXR1cmVzIjp0cnVlfX19XSwidGltZXNjYWxlIjoiIiwiZGlzcGxheUxlZ2VuZCI6dHJ1ZSwiYWxpZ25Nb250aCI6dHJ1ZX0%3D)  
  - Originally source from [Inside Airbnb](
http://insideairbnb.com/)
  
[New York City Airbnb Open Data](https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data)  
  - Dataset available under Creative Commons CC0: Public Domain License, 2019  
  - Uploaded to Kaggle by user [Dgomonov](https://www.kaggle.com/dgomonov)
  - Originally sourced from [Inside Airbnb](
http://insideairbnb.com/)
  
[U.S. Airbnb Open Data](https://www.kaggle.com/kritikseth/us-airbnb-open-data)  
  - Dataset available under Creative Commons CC0: Public Domain License, 2020  
  - Uploaded to Kaggle by user [Kritik Seth](https://www.kaggle.com/kritikseth)  
  - Originally sourced from [Inside Airbnb](
http://insideairbnb.com/)
    
  